In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
# Deps
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge


In [3]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()


## Load the data

In [4]:
# Load and preprocess the data
train_df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
train_df.head()


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [5]:
### Test dataset:
test_df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv")
test_df_targets = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv")

sample_submission = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv")
test_df


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,currently_scored
0,0,478,0,3753451.43,-1,0.999875,11548975.43,NaN,NaN,0.999875,22940.00,1.000050,9177.60,1.000000,26290,478_0_0,False
1,1,478,0,985977.11,-1,1.000245,3850033.97,NaN,NaN,0.999940,1967.90,1.000601,19692.00,1.000000,26290,478_0_1,False
2,2,478,0,599128.74,1,1.000584,4359198.25,NaN,NaN,0.999918,4488.22,1.000636,34955.12,1.000000,26290,478_0_2,False
3,3,478,0,2872317.54,-1,0.999802,27129551.64,NaN,NaN,0.999705,16082.04,1.000189,10314.00,1.000000,26290,478_0_3,False
4,4,478,0,740059.14,-1,0.999886,8880890.78,NaN,NaN,0.999720,19012.35,1.000107,7245.60,1.000000,26290,478_0_4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32995,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,26454,480_540_195,False
32996,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,26454,480_540_196,False
32997,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,26454,480_540_197,False
32998,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,26454,480_540_198,False


## PreProcessing for X Train y Train

In [6]:
def add_seconds_to_row_id(row_id, seconds):
    row_split = row_id.split("_")
    return row_split[0] + '_' + str(int(row_split[1]) + 60) + '_' + row_split[2]

In [7]:
train_df_2 = train_df.copy()
train_df_2['row_id_60_secs_to_future'] = train_df_2.apply(lambda row: \
          add_seconds_to_row_id(row.row_id, 60), axis=1)
train_df_60_secs_to_future = train_df[["row_id", "wap"]].copy()
train_df_60_secs_to_future.columns = train_df_60_secs_to_future.columns.map(lambda col: \
                              col+'_60_secs_to_future')
train_df_2 = pd.merge(train_df_2, train_df_60_secs_to_future, \
                            on=['row_id_60_secs_to_future'], how='left')
train_df_2["index_diff"]=(train_df_2["wap_60_secs_to_future"]/train_df_2["wap"])-train_df_2["target"]/10000

In [8]:
y = train_df_2.dropna()[['index_diff','wap_60_secs_to_future', 'target']]
X = train_df_2.dropna().drop(['index_diff','target', 'row_id','far_price', 'near_price','row_id_60_secs_to_future', 'wap_60_secs_to_future','time_id'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,)
y_train_index = y_train['index_diff']
y_test_index = y_test['index_diff']
y_train_wap = y_train['wap_60_secs_to_future']
y_test_wap = y_test['wap_60_secs_to_future']

In [9]:
X_cols = X.columns
imputer = SimpleImputer(missing_values=np.nan, strategy='mean').fit(X_train[X_cols])

## Building model

In [10]:
# Train the models
model_index_diff = Ridge(alpha=0.375)
model_wap_60_secs_to_future = LinearRegression()

In [11]:
model_index_diff.fit(X_train, y_train_index)
model_wap_60_secs_to_future.fit(X_train, y_train_wap)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.19549e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


LinearRegression()

In [12]:
def conj_model_predict(model_index, model_wap, X_test):
    storage_df = pd.DataFrame()
    storage_df["wap"] = X_test["wap"]
    storage_df["index_diff"] = model_index.predict(X_test)
    storage_df["wap_60_secs_to_future"] = model_wap.predict(X_test)
    return (10000*(storage_df["wap_60_secs_to_future"]/X_test["wap"]-storage_df["index_diff"])).to_numpy(dtype='float32')

In [13]:
y_pred = conj_model_predict(model_index_diff, model_wap_60_secs_to_future, X_test)
y_pred

array([ 0.08655965, -0.06191594, -0.44733703, ..., -0.28914472,
       -0.21908912, -0.9072781 ], dtype=float32)

## Evaluation

In [14]:
mae = mean_absolute_error(y_pred, y_test['target'])
print("Mean Absolute Error: {}".format(mae))

Mean Absolute Error: 5.5984524470947195


### Prediction on the real test data

In [15]:

test_preds = conj_model_predict(model_index_diff, model_wap_60_secs_to_future, test_df[X_cols])
test_preds

array([-0.4166315,  1.3969814,  1.8239378, ...,  0.7689604,  0.530404 ,
       -0.499064 ], dtype=float32)

In [16]:
# pd.DataFrame({'row_id':sample_submission["row_id"],'target':test_preds}).to_csv('submission.csv', index=False)

In [17]:
test, revealed_targets, test_preds = next(iter_test)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [18]:
env.predict(test_preds)

In [19]:
revealed = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')


In [20]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
    test_df_2=pd.DataFrame(imputer.transform(test[X_cols]))
    test_df_2.columns=test[X_cols].columns
    test_df_2.index=test[X_cols].index
    sample_prediction['target'] = conj_model_predict(model_index_diff, model_wap_60_secs_to_future, test_df_2[X_cols])
    env.predict(sample_prediction)
    counter += 1

   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                 10      3771174.79   
1         1      478                 10       967084.13   
2         2      478                 10       712904.65   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         1.000050   11550982.93        NaN   
1                       -1         0.999991    3868926.94        NaN   
2                        1         1.000482    4245422.33        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size       wap    row_id  \
0         NaN   1.000050  17208.00   1.000224  21456.38  1.000127  478_10_0   
1         NaN   0.999940   1967.90   1.000346  20671.35  0.999975  478_10_1   
2         NaN   1.000123  14833.68   1.000636  54092.56  1.000234  478_10_2   

   currently_scored  
0             False  
1             False  
2             False  
  stock_id  date_id  seconds_in_bucket revealed_target revealed_date_